In [30]:
from zumi.util.camera import Camera
from zumi.util.screen import Screen
from zumi.personality import Personality
from zumi.protocol import Note 

from zumi.zumi import Zumi
import cv2

import IPython.display
import PIL.Image
import time
import math
import pyzbar.pyzbar as pyzbar
import numpy as np

screen = Screen()
zumi = Zumi()

#width = 640*2
#height = 480*2

#width = 320
#height = 240

#width = 160 
#height = 120

#width = 80
#height = 60

personality = Personality(zumi, screen)
camera = Camera(width,height)
camera.start_camera()

zumi.mpu.calibrate_MPU()
zumi.reset_gyro()

heading = 0
celebrated = False

screen.blink()

        
#take in the frame and the width the frame
def follow_qr(frame,w_px):
    
    # Run a function that looks for codes in that frame
    decodedObjects = pyzbar.decode(frame) 
    
    # If the code finds more than one code...
    if len(decodedObjects) > 0:
        
        # For each QR code...
        for obj in decodedObjects: 
            
            #move Zumi depending on location of object
            width_of_qr = obj.rect[2]
            height_of_qr = obj.rect[3]
            cx_of_qr = obj.rect[0]+ width_of_qr/2
            x_middle_of_screen = w_px/2

            area_rectangle = width_of_qr*height_of_qr
            distance = calc_distance_to_qr(area_rectangle)
            
            error = (x_middle_of_screen-cx_of_qr)
            error_drive(error,distance)
            
            print("distance",distance)
            print("Message: ", obj.data.decode("utf-8"))
            print("error ", error)
    else:
        #if no QR code found, stop driving
        zumi.stop()

def calc_distance_to_qr(qr_area):
    #distance = 2594*qr_area**(-0.5)*1.3
    length_of_qr = 3 #3 inches
    area_of_object = length_of_qr**2
    area_detected = qr_area
    k = 0.96 #some constant which we can vary to get better results
    distance = ( k*(area_of_object*width*height)/area_detected )**(1/2)
    return distance
        
        
def error_drive(error, distance, error_threshold=5):
    global heading,celebrated
    #these are in inches
    min_distance = 4
    max_distance = 7
    speed = 20
    
    #if zumi is on the left or right of the object
#     if abs(error) > error_threshold:
#         celebrated = False
#         heading = heading + error/10
#         zumi.turn(heading,0.3)
    #if zumi is on the left or right of the object
    if abs(error) > error_threshold:
        
        val = int(abs(error)*0.01+2)
        print(val)
        zumi.control_motors(-1*val,val)
    
#     if error > error_threshold: 
#         #zumi.turn_left(0.1*abs(error),0.1)
#         zumi.control_motors(error,-1*error)
        
#     elif error < -1*error_threshold: 
#         zumi.turn_right(0.1*abs(error),0.1)


#     #if object is in the center and its far away
#     elif abs(error) < error_threshold and distance > max_distance:
#         distance_error = distance - max_distance
#         zumi.forward(speed*distance_error/10,0.2)
    
#     #if object is in the center but really close
#     elif abs(error) < error_threshold and distance < min_distance:
#         distance_error = min_distance - distance
#         zumi.reverse(speed*distance_error/10,0.2)
        
    else:
        zumi.stop()
#         if celebrated == False:
#             zumi.play_note(Note.C4,1)
#             zumi.play_note(0,0)
#             screen.happy()
#             celebrated = True
        
        
try: 
    while True:
        #update the angle traveled
        zumi.update_angles()
    
        #keep track of the time
        time_start = time.time()
        
        #take a picture
        image = camera.capture() 
        
        #follow the qr code
        follow_qr(image,width)
        
         #convert the image for displaying
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Convert it to gray
        ret,black_white = cv2.threshold(gray,100,255,cv2.THRESH_BINARY)
        
#         #display the video
        IPython.display.display(PIL.Image.fromarray(black_white))
        print("FPS", 1/(time.time()-time_start))
        print("heading ",heading)
        print("angle ", zumi.angle_list[2])
        IPython.display.clear_output(wait=True) 

finally:
    camera.close()
    zumi.stop()
    print("done")

 Starting Zumi 
Pi Zero I2C is available
Gyro offset file already exists.
Verified Pi Zero is the same
Gyroscope previously calibrated
Zumi board detected
Compass detected
OLED Screen detected
Gyroscope & Accelerometer detected
Starting PiCamera


PiCameraMMALError: Failed to enable connection: Out of resources

In [ ]:
zumi.stop()